In [1]:
import pandas as pd
import string
import re
import nltk
import time


stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()


df = pd.concat([pd.read_csv('data/data_{}.csv'.format(a)) for a in range(10)])
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df.head()


(9656817, 2)
(8961258, 2)


,Tweet,Category
0,i love lebron. http://bit.ly/PdHur,0
1,"@ludajuice Lebron is a Beast, but I'm still ch...",0
2,@wordwhizkid Lebron is a beast... nobody in th...,0
3,Bill Simmons in conversation with Malcolm Glad...,0
4,playing with Java and the Twitter API,0


In [2]:
# retweets?
df['Tweet'] = df['Tweet'].astype(str)
df['Retweet'] = df['Tweet'].apply(lambda x:x[:2]=='RT')

df['Retweet'].sum()

647511

In [3]:
import numpy as np
np.sum(df['Category'])

2365553

In [4]:

print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8961258
Russian 2365553


In [5]:

def preprocess_text(series):
     # @somone to @
    lst = []
    for text in series:
         # remove hashtags, etc for grammer checking
        text = ' '.join(['' if '@' in word else word for word in text.split()])
        text = ' '.join(['' if 'http' in word or 'www.' in word else word for word in text.split()])
        text = ' '.join(['' if '#' in word else word for word in text.split()])
        # remove all other punctuation
        text = ''.join([word.lower() for word in text if word not in string.punctuation.replace('@','').replace('#','').replace('_','')])

        # the real work, pos tagging to reduce Nouns and other things that don't matter
        tokens = [a[1] for a in nltk.pos_tag(text.split())]
#         for a in nltk.pos_tag(text.split()):
#             # keep these as-is for reasons
#             if '_link_' in a[0] or '_at_someone_' in a[0] or '_hashtag_' in a[0]:
#                 tokens.append(a[0])
# #             # lemmatize these to reduce parts of speach issues
# #             elif 'IN' in a[1] or 'V' in a[1] or 'DT' in a[1] or 'CC' in a[1] or 'MD' in a[1] or 'PDT' in a[1]:
# #                 tokens.append(nltk.PorterStemmer().stem(a[0]))
#             else:
#                 tokens.append(a[1])
       
        lst.append(' '.join(tokens))
    return lst 

In [6]:
# test it...
test = df.head().copy()
test['Processed'] = preprocess_text(test['Tweet'])
test.head()

,Tweet,Category,Retweet,Processed
0,i love lebron. http://bit.ly/PdHur,0,False,NN VBP NN
1,"@ludajuice Lebron is a Beast, but I'm still ch...",0,False,NN VBZ DT NN CC NN RB VBG CD DT NN DT NN
2,@wordwhizkid Lebron is a beast... nobody in th...,0,False,NN VBZ DT NN NN IN DT NN VBZ RB JJ
3,Bill Simmons in conversation with Malcolm Glad...,0,False,NN NNS IN NN IN NN NN
4,playing with Java and the Twitter API,0,False,VBG IN NN CC DT NN NN


In [7]:
import pickle
start = time.time()
df['Processed'] = preprocess_text(df['Tweet'])
end = time.time()
print('It took {} s to process/pos-tag the tweets.'.format(end-start))
pickle.dump( df, open( "files/df.pickle", "wb" ) )
df.head()

It took 9274.957566261292 s to process/pos-tag the tweets.


,Tweet,Category,Retweet,Processed
0,i love lebron. http://bit.ly/PdHur,0,False,NN VBP NN
1,"@ludajuice Lebron is a Beast, but I'm still ch...",0,False,NN VBZ DT NN CC NN RB VBG CD DT NN DT NN
2,@wordwhizkid Lebron is a beast... nobody in th...,0,False,NN VBZ DT NN NN IN DT NN VBZ RB JJ
3,Bill Simmons in conversation with Malcolm Glad...,0,False,NN NNS IN NN IN NN NN
4,playing with Java and the Twitter API,0,False,VBG IN NN CC DT NN NN
